In [1]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=07835a0ec121cf25f816f63096e6b9a1dcf296cc64033e56bb5d68fcb850b32f
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
      coverpage=soup.find_all('loc')
      for link in coverpage:
        if link.get_text().find('internacional')>=0 :
          rows.append(link.get_text()) 
        else :
          if link.get_text().find('espana')>=0:
            rows.append(link.get_text()) 
    except:
      rows = 'error'
      
    return rows

In [23]:
extract_urls_topic('https://www.elperiodicomediterraneo.com/sitemap_contents_2022_08_2a4b0_001.xml')

['https://www.elperiodicomediterraneo.com/internacional/2022/08/05/sergio-massa-nuevo-superministro-sacar-73107501.html',
 'https://www.elperiodicomediterraneo.com/internacional/2022/08/05/israel-mata-numero-yihad-islamica-73106956.html',
 'https://www.elperiodicomediterraneo.com/espana/2022/08/05/ley-vivienda-paralizada-pese-urgencia-73105829.html',
 'https://www.elperiodicomediterraneo.com/sociedad/2022/08/05/espana-suma-365-casos-viruela-73105238.html',
 'https://www.elperiodicomediterraneo.com/espana/2022/08/05/detienen-primera-vez-combatientes-daesh-73101275.html',
 'https://www.elperiodicomediterraneo.com/espana/2022/08/05/sanchez-recuerda-trece-rosas-quisieron-73103881.html',
 'https://www.elperiodicomediterraneo.com/espana/2022/08/05/muerte-corbata-73103678.html',
 'https://www.elperiodicomediterraneo.com/espana/2022/08/05/fiscalia-barcelona-envia-madrid-diligencias-73102836.html',
 'https://www.elperiodicomediterraneo.com/espana/2022/08/05/ue-traslada-mil-pacientes-ucranianos-

In [24]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

url_str=''

try:
  coverpage = requests.get('https://www.elperiodicomediterraneo.com/sitemap_index_2a4b0.xml').content
  soup = BeautifulSoup(coverpage, 'html.parser')
  
  rows =[]
  coverpage=soup.find_all('loc')
  for link in coverpage:
    rows.append(link.get_text()) 
except:
  rows = 'error'

df = pd.DataFrame(range(0,48),columns = ['page'])
df['page']=rows
df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/elperiodicomediterraneo/url.csv',index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


                                                                                 page  \
0      https://www.elperiodicomediterraneo.com/sitemap_contents_2022_08_2a4b0_001.xml   
1      https://www.elperiodicomediterraneo.com/sitemap_contents_2022_08_2a4b0_001.xml   
2      https://www.elperiodicomediterraneo.com/sitemap_contents_2022_08_2a4b0_001.xml   
3      https://www.elperiodicomediterraneo.com/sitemap_contents_2022_08_2a4b0_001.xml   
4      https://www.elperiodicomediterraneo.com/sitemap_contents_2022_08_2a4b0_001.xml   
...                                                                               ...   
30646  https://www.elperiodicomediterraneo.com/sitemap_contents_2008_09_2a4b0_001.xml   
30647  https://www.elperiodicomediterraneo.com/sitemap_contents_2008_09_2a4b0_001.xml   
30648  https://www.elperiodicomediterraneo.com/sitemap_contents_2008_09_2a4b0_001.xml   
30649  https://www.elperiodicomediterraneo.com/sitemap_contents_2008_09_2a4b0_001.xml   
30650  https://www.el